<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#A-tool-to-visualize-trajectory-frames:-ViewManager" data-toc-modified-id="A-tool-to-visualize-trajectory-frames:-ViewManager-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A tool to visualize trajectory frames: ViewManager</a></span><ul class="toc-item"><li><span><a href="#Example---show-PT-structures-with-5-largest-and-5-smallest-potential-energies" data-toc-modified-id="Example---show-PT-structures-with-5-largest-and-5-smallest-potential-energies-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example - show PT structures with 5 largest and 5 smallest potential energies</a></span></li><li><span><a href="#Example---plot-an-eigenvector-of-SqRA" data-toc-modified-id="Example---plot-an-eigenvector-of-SqRA-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Example - plot an eigenvector of SqRA</a></span></li></ul></li><li><span><a href="#Classify-full-trajectories-in-cells-of-FullGrid" data-toc-modified-id="Classify-full-trajectories-in-cells-of-FullGrid-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Classify full trajectories in cells of FullGrid</a></span></li><li><span><a href="#Assigning-trajectories-to-a-FullGrid-cell" data-toc-modified-id="Assigning-trajectories-to-a-FullGrid-cell-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Assigning trajectories to a FullGrid cell</a></span><ul class="toc-item"><li><span><a href="#Test:-assigning-PT-to-a-FullGrid" data-toc-modified-id="Test:-assigning-PT-to-a-FullGrid-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Test: assigning PT to a FullGrid</a></span></li><li><span><a href="#Assigning-a-real-trajectory-fo-a-FullGrid" data-toc-modified-id="Assigning-a-real-trajectory-fo-a-FullGrid-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Assigning a real trajectory fo a FullGrid</a></span></li></ul></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nglview as nv
import MDAnalysis as mda
import numpy as np
import time

from scipy.sparse import csr_array
from numpy.typing import NDArray
import matplotlib

from molgri.molecules.transitions import SimulationHistogram, MSM, SQRA
from molgri.plotting.molecule_plots import TrajectoryPlot
import pandas as pd
import matplotlib.pyplot as plt
from molgri.molecules.parsers import FileParser, ParsedEnergy, XVGParser

from molgri.paths import PATH_OUTPUT_PT, PATH_OUTPUT_LOGGING, PATH_OUTPUT_ENERGIES
from molgri.space.fullgrid import FullGrid
from molgri.space.utils import k_argmin_in_array, k_argmax_in_array
from molgri.plotting.widgets import ViewManager, sync_all_views, display_all_views

## A tool to visualize trajectory frames: ViewManager

- in Jupyter, using nglview
- input: mda.Universe and one or more indices along the trajectory
- output: frames plotted sequentially or overlapping, color/opacity indicating a property

### Example - show PT structures with 5 largest and 5 smallest potential energies

In [3]:
# EXAMPLE - 
sh_pt = SimulationHistogram("H2O_H2O_0179", is_pt=True, second_molecule_selection="bynum 4:6")


# display
vm = ViewManager(sh_pt.trajectory_universe)
vm.fresh_view()

# magnitudes

magnitudes = sh_pt.get_magnitude_energy("Potential")


# 5 structures with largest energies
num_extremes = 5
argmax_index = k_argmax_in_array(magnitudes, num_extremes)
view1 = vm.plot_frames_overlapping(argmax_index, color_magnitudes=magnitudes, opacities=0.5)

# 5 structures with smallest energies
vm.fresh_view()
argmin_index = k_argmin_in_array(magnitudes, num_extremes)
view2 = vm.plot_frames_overlapping(argmin_index, color_magnitudes=magnitudes, opacities=0.5)

# show both
sync_all_views([view1, view2])
display_all_views([view1, view2])

Box(children=(NGLWidget(), NGLWidget()), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100…

### Example - plot an eigenvector of SqRA

In [ ]:
# EXAMPLE - eigenvector

# display
vm = ViewManager(pce.u)
vm.fresh_view()

# magnitudes
mch = MplColorHelper()
magnitudes = pce.get_magnitude_ith_eigenvector(2)

colors = np.array([mch.get_hex(mag) for mag in magnitudes])


# 5 largest, 5 smallest
num_extremes = 20
argmin_index = k_argmin_in_array(magnitudes, num_extremes)
argmax_index = k_argmax_in_array(magnitudes, num_extremes)
both_index = [*argmax_index, *argmin_index]

print(magnitudes[both_index])

vm.plot_frames_overlapping(both_index, colors=colors[both_index], opacities=[0.5]*len(both_index))
vm.view

In [ ]:
# not PT
my_path = "/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_2000/"

u_traj = mda.Universe(f"{my_path}H2O_H2O_0095.gro", f"{my_path}fitted_output.xtc")

fg = FullGrid(o_grid_name="12", b_grid_name="8", t_grid_name="linspace(0.2, 1, 20)", use_saved=True)



my_path = "/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_2000/"
topology = f"{my_path}H2O_H2O_0095.gro"
coordinates = f"{my_path}fitted_output.xtc"
energy = f"{my_path}full_energy.xvg"

# preparing the parsed trajectory
my_parser = XVGParser(energy)
pe = my_parser.get_parsed_energy()
pt_parser = FileParser(
    path_topology=topology,
    path_trajectory=coordinates)
parsed_trajectory = pt_parser.get_parsed_trajectory(default_atom_selection="bynum 4:6")
parsed_trajectory.energies = pe


sm = SimulationHistogram(parsed_trajectory, fg)
my_array1 = sm.get_all_assignments()

cell_74 = np.where(my_array1==74)[0]
print(cell_74)

#view = nv.show_mdanalysis(u_traj)
#view.add_unitcell()
#view
vm = ViewManager(u_traj)
vm.fresh_view()
vm.plot_frames_overlapping(cell_74)

In [ ]:
all_energies = parsed_trajectory.energies.get_energies("Potential")
lowest_E = k_argmin_in_array(all_energies, 20)
print(lowest_E)
vm.fresh_view()
vm.plot_frames_overlapping(lowest_E)

## Classify full trajectories in cells of FullGrid

In [ ]:
from molgri.molecules.writers import PtIOManager

from MDAnalysis.coordinates.memory import MemoryReader
from MDAnalysis.analysis.base import AnalysisFromFunction

def _extract_universe_second_molecule(original_universe, selection_criteria):
    m2 = original_universe.select_atoms(selection_criteria)

    coordinates = AnalysisFromFunction(lambda ag: ag.positions.copy(), m2).run().results['timeseries']
    u2 = mda.Merge(m2)
    u2.load_new(coordinates, format=MemoryReader)
    return u2


def assign_trajectory_2_quaternion_grid(trajectory_universe: mda.Universe, m1_name, m2_name, b_grid_name,
                                       second_molecule_selection):
    
    # create PT on quaternion-only grid
    manager = PtIOManager(m1_name, m2_name, o_grid_name="1", b_grid_name=b_grid_name, 
                          t_grid_name="[0.1]")
    my_pt = manager.construct_pt()
    my_pt_name = manager.get_name()
    pt_sec_mol_universe = mda.Universe(f"{PATH_OUTPUT_PT}{my_pt_name}.gro", f"{PATH_OUTPUT_PT}{my_pt_name}.xtc")
    
    # in the real and pt trajectory, extract the second molecule and center it without rotating
    trajectory_universe_m2 = _extract_universe_second_molecule(trajectory_universe, second_molecule_selection)
    pt_universe_m2 = _extract_universe_second_molecule(pt_sec_mol_universe, second_molecule_selection)
    # move them to center - curently doing that later
    #workflow = [mda.transformations.center_in_box(real_traj_sec_mol.atoms, center="mass", point=(0, 0, 0))]
    #real_traj_sec_mol.trajectory.add_transformations(*workflow)
    
    # calculate RMSD between both
    total_results = []
    for i, ts in enumerate(pt_universe_m2.trajectory):
        results = []
        for j, ts2 in enumerate(trajectory_universe_m2.trajectory):
            results.append(mda.analysis.rms.rmsd(real_traj_sec_mol.trajectory[j].positions, 
                                                 pt_sec_mol.trajectory[i].positions,
                                                center=True, weights=real_traj_sec_mol.atoms.masses))
        total_results.append(results)
    total_results = np.array(total_results)
    clases = np.argmin(total_results, axis=0)
    return clases

    

my_cl = assign_trajectory_2_quaternion_grid(u_traj, "H2O", "H2O", "8", "bynum 4:6")
print(np.unique(my_cl, return_counts=True))

In [ ]:
selected_class = 4

vm = ViewManager(pt_sec_mol)
vm.fresh_view()
vm.plot_ith_frame(selected_class)


In [ ]:
for selected_class in range(8):
    vm = ViewManager(real_traj_sec_mol)
    vm.fresh_view()
    vm.plot_frames_overlapping(np.where(clases==selected_class)[0][::20])
    display(vm.view)

In [ ]:
# TODO: enable different colormaps
# TODO: enable expressing magnitude as opacity
# TODO: enable plotting only the most extrem values (most + and - for eigenvector)
# TODO: plot eigenvectors and see if they are sensible
# TODO: also plot 1D eigenvectors sorted by orientation/position
# TODO: network flow plot for strongest rates?

# TODO: make transition matrix determination faster and try it for a large matrix
# TODO: HF forcefield and calculations
# TODO: enable the same for plotting for simulations (real trajectories)

## Assigning trajectories to a FullGrid cell
- input: (pseudo)trajectory and a FullGrid of choice
- assign structures to position, orientation or full classes (cells of FullGrid) 


### Test: assigning PT to a FullGrid

- if assigning to its own fg, assignments should be 1:1
- if assigning to a smaller fg, an uniform number per cell is expected

In [ ]:
evaluation_fg = FullGrid("40", "42", "linspace(0.2, 1.5, 10)")

# this pt was generated with fg = FullGrid("40", "42", "linspace(0.2, 1.5, 10)")
my_pt_name = "H2O_H2O_0179"

sh = SimulationHistogram(my_pt_name, full_grid=evaluation_fg, second_molecule_selection="bynum 4:6")
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
assignments = sh.get_position_assignments()
natural_numbers = np.array(range(len(sh.full_grid.get_position_grid_as_array())))
natural_numbers = np.repeat(natural_numbers, sh.full_grid.b_rotations.get_N())

assert np.all(assignments == natural_numbers)

assignments = sh.get_quaternion_assignments()
natural_numbers = np.array(range(sh.full_grid.b_rotations.get_N()))
natural_numbers = np.tile(natural_numbers, len(sh.full_grid.get_position_grid_as_array()))

for el1, el2 in zip(assignments, natural_numbers):
    print(el1, el2)
    #pass

#assignments = sh.get_full_assignments()
#natural_numbers = np.array(range(len(assignments)))

#for el1, el2 in zip(assignments, natural_numbers):
#    print(el1, el2)

### Assigning a real trajectory fo a FullGrid

In [ ]:
my_fg = FullGrid("8", "12", "[0.3, 0.6, 0.9, 1.2]")
sh_traj = SimulationHistogram("H2O_H2O_0095_2000", full_grid=my_fg, second_molecule_selection="bynum 4:6")


In [ ]:
# most populated cells 
assignments = sh_traj.get_full_assignments()
my_indices, my_counts = np.unique(assignments.astype(int), return_counts=True)
most_populated_cells = my_indices[k_argmax_in_array(my_counts, 5)]
vm = ViewManager(sh.trajectory_universe)
for selected_class in most_populated_cells:
    vm.fresh_view()
    vm.plot_frames_overlapping(np.where(assignments==selected_class)[0][::10])
    display(vm.view)

In [ ]:
# are the most populated cells also the ones with lowest average energy?
# maybe not because the size of the cell strongly affects the population
all_popular_frames = []
full_assignments = sh_traj.get_full_assignments().astype(int)
my_indices, my_counts = np.unique(full_assignments, return_counts=True)
most_populated_cells = my_indices[k_argmax_in_array(my_counts, 3)]
for populated_cell in most_populated_cells:
    belongs_to_this = np.where(sh_traj.get_full_assignments()==populated_cell)[0]
    all_popular_frames.extend(belongs_to_this)

cell_volumes = np.array(sh_traj.full_grid.get_total_volumes())
volume_per_assignment = cell_volumes[full_assignments]

pot_energy = energies.get_energies("Potential")/volume_per_assignment
print("Popular frames energy", np.min(pot_energy[all_popular_frames]), np.average(pot_energy[all_popular_frames]),
     np.max(pot_energy[all_popular_frames]))
print("All frames energy", np.min(pot_energy), np.average(pot_energy),
     np.max(pot_energy))

In [ ]:
from molgri.plotting.transition_plots import TransitionPlot
my_msm = MSM(sh, energies)
my_tp = TransitionPlot(my_msm)
my_tp.plot_heatmap(save=False)

In [ ]:
my_tp.plot_its(save=False)